<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ADK_LLAMA3_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test Model and run Inference

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q sentence_transformers
!pip install -U flash-attn --no-build-isolation --quiet
!pip install -q tqdm
!pip install -q colab-env
import colab_env

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline, BitsAndBytesConfig

peft_model_id = "/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

In [5]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [6]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer

In [8]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')

if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]

print(f"Generated Answer:\n{ganswer}")

Generated Answer:
antral follicle count


In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers import pipeline
from datasets import load_dataset

# Load a pre-trained sentence embedding model
model_embedding = SentenceTransformer('all-mpnet-base-v2')

# Load your test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

# Create the generation pipeline
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def evaluate(sample):
    prompt = sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.7,
                                  top_k=50, top_p=0.7, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer = outputs[0]['generated_text'][len(prompt):].strip()
    qc0 = str(ganswer).find('[/INST]')
    qc1 = str(ganswer).find('[INST]')
    ganswer = str(ganswer)[qc0 + 8:qc1 - 8]

    oanswer = str(sample['label'])
    oanswer = oanswer[2:len(oanswer) - 2]

    if len(oanswer) == 0:
        oanswer = 'Not possible to get or use'

    #print('\n\n')
    #print(f"Question: {prompt}")
    #print(f"Generated Answer: {ganswer}")
    #print(f"Original Answer: {oanswer}")

    # Generate sentence embeddings
    embedding1 = model_embedding.encode(ganswer, convert_to_tensor=True)
    embedding2 = model_embedding.encode(oanswer, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_sim = util.cos_sim(embedding1, embedding2).item()

    # Set a similarity threshold (adjust as needed)
    if cosine_sim >= 0.5:
        #print("Match")
        return 1
    else:
        return 0


In [10]:
success_rate = []
number_of_eval_samples = 10  # Adjust the number of samples as needed

# Evaluate the model
for n in tqdm(range(number_of_eval_samples)):
    s = eval_dataset[n]
    success_rate.append(evaluate(s))

100%|██████████| 10/10 [02:49<00:00, 16.95s/it]


In [22]:
# Calculate accuracy
accuracy = sum(success_rate) / len(success_rate)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7


## ADK TUTORIAL

In [ ]:
!pip install google-cloud-aiplatform[adk,agent_engines] -q
!pip install colab-env -q

In [2]:
# Import libraries from the Agent Development Kit
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()

In [ ]:
from google.cloud import aiplatform
import colab_env
import os

# Get project details from environment variables
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
REGION = os.environ.get("GOOGLE_CLOUD_REGION")
SERVICEACCOUNT = os.environ.get("GOOGLE_CLOUD_SERVICEACCOUNT")
PROJECT_NUMBER = os.environ.get("GOOGLE_CLOUD_PROJECT_NUMBER")
BUCKET_NAME = os.environ.get("GOOGLE_CLOUD_BUCKET_NAME")
STAGING_BUCKET = f"gs://{BUCKET_NAME}/staging"


GOOGLE_GENAI_USE_VERTEXAI=1
GOOGLE_CLOUD_PROJECT=PROJECT_ID
GOOGLE_CLOUD_LOCATION=REGION
MODEL='gemini-2.0-flash-001'

In [5]:
# Create a basic agent with name, description, and instructions only
basic_agent = Agent(
    model=MODEL,
    name="medical_agent",
    description="This agent responds to medical inquiries.",
    #instruction="Respond to user questions accurately and concisely based on the information provided in the query.", # Changed instruction
    instruction = "Identify and extract the specific term that the query is defining. Provide only that term as your answer.",
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [6]:
AGENT_APP_NAME = 'medical_agent'

def send_query_to_agent(agent, query):
  session = session_service.create_session(app_name=AGENT_APP_NAME,
                                           user_id='user',)

  content = types.Content(role='user', parts=[types.Part(text=query)])
  runner = Runner(app_name=AGENT_APP_NAME,
                agent=agent,
                artifact_service=artifact_service,
                session_service=session_service)
  events = runner.run(user_id='user',
                    session_id=session.id,
                    new_message=content,)

  for _, event in enumerate(events):
    is_final_response = event.is_final_response()
    if is_final_response:
        final_response = event.content.parts[0].text
        print(f"Final Response: {final_response}")

In [7]:
query2="the aim of this T0 was to evaluate the effect of a single dose of mg of the selective alpha adrenergic agonist clonidine on the cardiovascular and hormonal responses to a standardized mental arithmetic task in healthy volunteers the T0 was performed in a doubleblind randomized crossover design in healthy volunteers mean age years mean blood pressure mmhg mean heart rate bpm the subjects were studied on two occasions separated by a week interval on one occasion they received mg of clonidine and on the other occasion they received PL the order of the two treatments was randomized the subjects"

query22="while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in summary dor is found to be a clinical challenge in the practice of fertility care with controversial countermeasures to prevent or treat the condition nevertheless some promising measure such as oocyte embryo and tissue cryopreservation ovarian transplantation dietary supplementation and the transfer of mitochondria have offered hopes towards ameliorating the burden of dor this review attempts to discuss dor from different perspectives and summarize some existing hopes in clinical practice"

query3 = "In the context of ovarian reserve, what is the full term represented by the abbreviation AFC?"


In [8]:
print('\n\n')
print(f"Query:\n{query2}")
event=send_query_to_agent(basic_agent,  query2)
print('\n\n')

print(f"Query:\n{query22}")
event=send_query_to_agent(basic_agent,  query22)
print('\n\n')

print(f"Query:\n{query3}")
event=send_query_to_agent(basic_agent,  query3)
print('\n\n')




Query:
the aim of this T0 was to evaluate the effect of a single dose of mg of the selective alpha adrenergic agonist clonidine on the cardiovascular and hormonal responses to a standardized mental arithmetic task in healthy volunteers the T0 was performed in a doubleblind randomized crossover design in healthy volunteers mean age years mean blood pressure mmhg mean heart rate bpm the subjects were studied on two occasions separated by a week interval on one occasion they received mg of clonidine and on the other occasion they received PL the order of the two treatments was randomized the subjects
Final Response: Clonidine




Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior subo